In [4]:
import sys
sys.path.append('/mnt/home/larend/kfx/src')

In [5]:
from src.simon.LFP_spk_preprocessing.utilities_ketamine_analysis_v8 import *

ModuleNotFoundError: No module named 'scipy'

In [ ]:
whos

In [ ]:
pip install quantities

In [ ]:
pip install neo

In [ ]:
pip install ghostipy

In [ ]:
pip install mat73

In [ ]:
# You may need to uncomment the following line if running this
# notebook on Colab

import ghostipy as gsp
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import os
import h5py
import warnings
from urllib.request import urlopen
plt.rcParams['pdf.fonttype'] = 42

In [ ]:
from scipy.signal import butter, lfilter, freqz, iirnotch, welch 

# Filenames to analyze

In [ ]:
binPath = r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Ketamine_data\Neurotar_HPC_ketamine\SPK\electrophysiology\\'
binFullPath = Path(binPath)

In [ ]:
#nt_Path for behavior
#sync_path for sync pulse
#ksort_path

dlc_path = binFullPath.parents[1]

files = dlc_path.glob('**/*.mat')

sync_path=[]
nt_path=[]

for i in sorted(files):
    
    if 'sync' in str(i): 
        print(i)
        sync_path.append(i)
        
    if 'behavior' in str(i): 
        print(i)
        nt_path.append(i)

print(nt_path[-1])

In [ ]:
sync_path

In [ ]:

#Example of how to append two files together
dlc_path = binFullPath
ksort_HPC_path=[]
ksort_PFC_path=[]

files = os.listdir(dlc_path)
for i in sorted(files):
            
    if 'HPCprobe' in str(i): 
        ksort_HPC_path.append(Path(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Ketamine_data\Neurotar_HPC_ketamine\SPK\electrophysiology\\' + i))
        print(i)
#     if 'PFCprobe' in str(i): 
#         ksort_PFC_path.append(Path(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Ketamine_data\Neurotar_HPC_ketamine\SPK\electrophysiology\\' + i))      
#         print(i)
        
print(ksort_HPC_path)

# Behavior

## Load behavior and sync files for aligment

In [ ]:
print(len(nt_path),len(sync_path),len(ksort_HPC_path))

In [ ]:
import mat73
ts_ephys_aln = []
nt_sync_aln = []
camera_sync_aln = []
ioFS = []
x = []
y = []
yaw = []
speed = []
sync_bool = []

for ipath in range(0,len(nt_path)):
    
    ipathsync = sync_path[ipath]
    ipathnt = nt_path[ipath]

    print('experiment',ipathsync)
    
    # raw files for sync and behavior
    sync_extracted = mat73.loadmat(ipathsync)
    behavior = loadmat(ipathnt)
    
    if np.abs(sync_extracted['samplesSyncAP_HPCprobe'].size - sync_extracted['samplesCamera'].size) < 1000:
        
        ts_ephys_aln.append(sync_extracted['samplesSyncAP_HPCprobe'])
        nt_sync_aln.append(sync_extracted['samplesNeurotarStrobe'])
        camera_sync_aln.append(sync_extracted['samplesCamera'])
        ioFS.append(sync_extracted['ioFS'])
        
        x.append(behavior['nt_x'][0])
        y.append(behavior['nt_y'][0])
        yaw.append(behavior['nt_alpha'][0])
        speed.append(behavior['nt_speed'][0])
        
        
        sync_bool.append(True)
        
    else:
        sync_bool.append(False)


#     if 'SAL' in ipathsync:

In [ ]:
ipath

In [ ]:
print(sync_bool)
sync_cond = (np.hstack(sync_bool))
print(sync_cond)

In [ ]:
idx_sync_ok = np.flatnonzero(sync_cond == True)
idx_sync_ok.shape

In [ ]:
#Already alligned
sync_path_matched = np.array(sync_path)[idx_sync_ok]
nt_path_matched = np.array(nt_path)[idx_sync_ok]
ksort_HPC_path_matched = np.array(ksort_HPC_path)[idx_sync_ok]
# ksort_PFC_path_matched = np.array(ksort_PFC_path)[idx_sync_ok]

In [ ]:
print(ksort_HPC_path_matched)

In [ ]:
%whos ndarray & list & DataFrame

In [ ]:
x_aln = []
y_aln = []
yaw_aln = []
speed_aln = []
duration = []
idx_camera = []
idx_camera_end = []
idx_nt = []
idx_nt_end = []

for ipath in range(0,len(x)):
    ipathsync = sync_path_matched[ipath]
    print('experiment',ipathsync)

    plt.plot(np.diff(ts_ephys_aln[ipath]))
    plt.plot(np.diff(nt_sync_aln[ipath]))
    plt.plot(np.diff(camera_sync_aln[ipath]))
    plt.show()

    print(camera_sync_aln[ipath])
    print(nt_sync_aln[ipath])
    
    _tmp_idx = []
    _tmp_idx = np.argwhere((camera_sync_aln[ipath] < nt_sync_aln[ipath][-1])==True)[-1]
    
    if ts_ephys_aln[ipath].size >= _tmp_idx:
        #Just to allign the camera with the sync, having the first frame of behavior and the last one
        idx_camera.append(np.argwhere((camera_sync_aln[ipath] > nt_sync_aln[ipath][0])==True)[0])
        idx_camera_end.append(np.argwhere((camera_sync_aln[ipath] < nt_sync_aln[ipath][-1])==True)[-1])
 
    if ts_ephys_aln[ipath].size < _tmp_idx:
        #Just to allign the camera with the sync, having the first frame of behavior and the last one
        idx_camera.append(np.argwhere((camera_sync_aln[ipath] > nt_sync_aln[ipath][0])==True)[0])
        idx_camera_end.append(np.array([ts_ephys_aln[ipath].size-1]).astype(int))
         
    
    print('camera',idx_camera[ipath],idx_camera_end[ipath])
    print(idx_camera_end[ipath]-idx_camera[ipath])


    print(ts_ephys_aln[ipath].size,nt_sync_aln[ipath].size,camera_sync_aln[ipath].size)


    duration.append(((ts_ephys_aln[ipath][idx_camera_end[ipath]] - ts_ephys_aln[ipath][idx_camera[ipath]]) /30000)[0])

    idx_nt.append(np.argwhere((nt_sync_aln[ipath] > camera_sync_aln[ipath][idx_camera[ipath]])==True)[0])
    idx_nt_end.append(np.argwhere((nt_sync_aln[ipath] < camera_sync_aln[ipath][idx_camera_end[ipath]])==True)[-1])
    print('nt',idx_nt[ipath],idx_nt_end[ipath])

    print(duration,(nt_sync_aln[ipath][idx_nt_end[ipath]] - nt_sync_aln[ipath][idx_nt[ipath]]) /ioFS)
    print('Deviation of',(duration[ipath] - (nt_sync_aln[ipath][idx_nt_end[ipath]] - nt_sync_aln[ipath][idx_nt[ipath]]) /ioFS[ipath]) * 1000, 'in miliseconds')
    print('Deviation at start',((camera_sync_aln[ipath][idx_camera[ipath]] - nt_sync_aln[ipath][idx_nt[ipath]])/ioFS[ipath]) * 1000,'in miliseconds')
    print('Deviation at end',((camera_sync_aln[ipath][idx_camera_end[ipath]] - nt_sync_aln[ipath][idx_nt_end[ipath]])/ioFS[ipath]) * 1000,'in miliseconds')

    
    
    ## align behavior

    x_aln.append(x[ipath][int(idx_nt[ipath][0]):int(idx_nt_end[ipath][0])])
    y_aln.append(y[ipath][int(idx_nt[ipath][0]):int(idx_nt_end[ipath][0])])
    yaw_aln.append(yaw[ipath][int(idx_nt[ipath][0]):int(idx_nt_end[ipath][0])])
    speed_aln.append(speed[ipath][int(idx_nt[ipath][0]):int(idx_nt_end[ipath][0])])

In [ ]:
## get lfp paths

In [ ]:
#Example of how to append two files together
path_lfp = Path(r'Z:\NeuroPix\Ketamine\\')
dlc_path = path_lfp

files = dlc_path.glob('**/*.imec1.lf.bin')
HPC_lfp_path=[]

# files = os.listdir(dlc_path)

for i in files:
    
    if 'De_nosing' in str(i):
        break    
        
    HPC_lfp_path.append(i)
    print(i)


In [ ]:
sorted(HPC_lfp_path)

In [ ]:
## split path by \
root_lfp = []
for i in HPC_lfp_path:

    arr = []
    arr = [ x.strip() for x in str(i).split('\\') ]
    
    naming_convention = []
    naming_convention = arr[3]
    new = np.char.replace(naming_convention,'-','_')

    print(naming_convention)
    root_lfp.append(new)


In [ ]:
## split path by \
HPC_lfp_path_matched = []
for i in ksort_HPC_path_matched:

    arr = []
    arr = [ x.strip() for x in str(i).split('\\') ]
    
    naming_convention = []
    naming_convention = arr[-1][:-9]
    new = []
    new = np.char.replace(naming_convention,'-','_')
    print(new)
    
    idx = root_lfp.index(new)
    
    print(HPC_lfp_path[idx])
    
    HPC_lfp_path_matched.append(HPC_lfp_path[idx])

In [ ]:
HPC_lfp_path_matched

In [ ]:
len(HPC_lfp_path_matched)

# about 2-3 min until this point !!

In [ ]:
# ## save paths

# import pickle

# outputlist = []
# outputlist = [HPC_lfp_path_matched, x_aln, y_aln, yaw_aln, speed_aln,ksort_HPC_path_matched]

# # to save "processed" aligned data
# with open(r'C:\Users\fentonlab\Desktop\Local_copy_Ketamine_analysis\HPC_path_info_behavior.file', 'wb') as f:
#     pickle.dump(outputlist, f, pickle.HIGHEST_PROTOCOL)
# print('saved!')


In [ ]:
## load behavior and path data

import pickle

HPC_lfp_path_matched, x_aln, y_aln, yaw_aln, speed_aln,ksort_HPC_path_matched = [],[],[],[],[],[]

# load data
with open(r'C:\Users\fentonlab\Desktop\Local_copy_Ketamine_analysis\HPC_path_info_behavior.file', 'rb') as f3:
    [HPC_lfp_path_matched, x_aln, y_aln, yaw_aln, speed_aln,ksort_HPC_path_matched] = pickle.load(f3)    
print('loaded!')

### Electrophysiology - Load units - HPC probe

In [ ]:
### file to load
ipath = 35

In [ ]:
time_bin = 0.01 # 1/1250 # in seconds; fine time resolution for phase-frequency analysis for PC and non-PCs analysis

fs = 30000

ksort_path = ksort_HPC_path_matched[ipath]
print('experiment',ksort_path)

cluster_info = []
cluster_info_path = ksort_path.joinpath('cluster_info.tsv')
cluster_group_path = ksort_path.joinpath('cluster_group.tsv')

try:
    cluster_info = pd.read_csv(cluster_info_path, sep='\t')
except:
    cluster_info = pd.read_csv(cluster_group_path, sep='\t')

# for screening look only at good units 
try:
    #cluster_info = cluster_info[cluster_info.group != 'noise']
    cluster_info = cluster_info[cluster_info.KSLabel == 'good']

#    cluster_info = cluster_info[cluster_info.group == 'good']
#    cluster_info = cluster_info[cluster_info.n_spikes > 100]
#    cluster_info = cluster_info[(cluster_info.group == 'good') * (cluster_info.depth > 1200)]

except:
    print('phy was not done')
    cluster_info = cluster_info[cluster_info.KSLabel == 'good']
    #cluster_info = cluster_info[cluster_info.KSLabel != 'noise']


spike_times = np.load(ksort_path.joinpath('spike_times.npy'))
spike_clusters = np.load(ksort_path.joinpath('spike_clusters.npy'))
templates = np.load(ksort_path.joinpath('templates.npy'))
spike_templates = np.load(ksort_path.joinpath('spike_templates.npy'))

# cid_channels = dict(zip(cluster_info.id.values, cluster_info.ch.values))
# cid_depth = dict(zip(cluster_info.id.values, cluster_info.depth.values))

df_spikes = pd.DataFrame({'times':spike_times.flatten()/30000.0, 
           'unit_id':spike_clusters.flatten(), 
           'templates':spike_templates.flatten() })
df_spikes.unit_id.unique().size



    # remove noise units
try:
    kslabel_d = dict(zip(cluster_info.id, cluster_info.group))
except:
    kslabel_d = dict(zip(cluster_info.cluster_id, cluster_info.KSLabel))

df_spikes['kslabel'] = df_spikes.unit_id.map(kslabel_d)
# remove noise units
df_spikes = df_spikes.dropna()
#     df_spikes_all.append(df_spikes)
print('allcells',df_spikes.unit_id.unique().size)

#     cids_to_keep = []
#     cids_to_keep = cluster_info[cluster_info['ch'].isin(ch_HPC_list[ipath])].cluster_id.values    
# #     print(cids_to_keep)
#     cids_to_keep_HPC_all.append(cids_to_keep)

start_offset = []
start_offset = int(ts_ephys_aln[ipath][idx_camera[ipath]])
print(start_offset,start_offset/fs)


## start with neurotar light recording
df_spk = []
df_spk = df_spikes.copy()
df_spk['spk'] = df_spikes['times']


df_spk= df_spk[(df_spk['spk'] >= (ts_ephys_aln[ipath][idx_camera[ipath]]/fs)[0]) & (df_spk['spk'] <= (ts_ephys_aln[ipath][idx_camera_end[ipath]]/fs)[0])]
df_spk.head()

df_spk['spk'] = df_spk['times'] - (ts_ephys_aln[ipath][idx_camera[ipath]]/fs)[0]
df_spk['spk'] 


df_units = []
df_units = bin_spk(df_spk, time_bin)  #0.01 is the time resolution to bin 0.01 s = 10ms


_inp_l = []
#     _inp_l = df_units[df_units['cids'].isin(cids_to_keep)].pivot('cids', 'times', 'hz').values[:, :]
_inp_l = df_units.pivot('cids', 'times', 'hz').values[:, :]

print('shape binned', _inp_l.shape)

spk_tmp = []

cids_good = []
cids_good = df_units.cids.unique()
print(cids_good.size)

channel_good = []
channel_good = cluster_info[cluster_info['cluster_id'].isin(cids_good)].ch.values    

In [ ]:
outputlist = []
outputlist = [_inp_l, cids_good, channel_good]

import pickle
# to save "processed" aligned data
with open(r'C:\Users\fentonlab\Desktop\Local_copy_Ketamine_analysis\spk_' + str(ipath) + '.file' , 'wb') as f:
    pickle.dump(outputlist, f, pickle.HIGHEST_PROTOCOL)
print('saved!')